# PlusMinus VQC — Blind Quantum Computing

Este notebook demonstra como treinar um modelo de Rede Neural Quântica (VQC)
usando o Qiskit Machine Learning e circuitos gerados a partir do dataset PlusMinus.

In [ ]:
#Instale as dependências 
!pip install qiskit qiskit-aer qiskit-machine-learning scikit-learn matplotlib numpy
!pip install qiskit-machine-learning
!pip install qiskit-algorithms pennylane

In [ ]:

import random
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import COBYLA

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error


# Controle de Aleatoriedade e Reprodutibilidade

In [ ]:

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
try:
    from qiskit_algorithms.utils import algorithm_globals
    algorithm_globals.random_seed = SEED
except ImportError:
    pass

# Backend

In [ ]:

noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['h','x','y','z','rx','ry','rz','s','sdg','t','tdg'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx','cz','swap'])

simulator = AerSimulator(noise_model=noise_model, shots=128, method='matrix_product_state')


# Função da Rede

In [ ]:
def enviar_circuito_por_epoca(circuito, epoch, num_qubits, circuit_depth):
    print(f"[Epoch {epoch+1}] Enviando circuito para a rede...")
    try:
        rede.application_layer.run_app(
            "BFK_BQC",
            alice_id=6,
            bob_id=0,
            num_qubits=num_qubits,
            scenario=2,
            circuit_depth=circuit_depth,
            circuit=circuito
        )
        print(f"[Epoch {epoch+1}] Envio concluído.")
    except Exception as e:
        print(f"[Epoch {epoch+1}] Erro ao enviar circuito: {str(e)}")

# Carregar e preparar o dataset Plus-Minus

In [ ]:

[ds] = qml.data.load("plus-minus")

X = ds.img_train
y = ds.labels_train
X_test = ds.img_test
y_test = ds.labels_test

from sklearn.decomposition import PCA
# Flatten das imagens: (n_amostras, altura*largura)
X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# PCA reduzindo para 4 dimensões (4 qubits)
pca = PCA(n_components=4, random_state=SEED)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)
#

X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))
scaler = MinMaxScaler((0,1))
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# VQC

In [ ]:
num_classes = len(np.unique(y))
num_qubits = X.shape[1]

feature_map = ZZFeatureMap(num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits, reps=1)
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

def interpret(x):
    return x % num_classes

sampler = Sampler(options={"backend": simulator})

qnn = SamplerQNN(
    circuit=qc,
    sampler=sampler,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=interpret,
    output_shape=num_classes,
)


vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=COBYLA(maxiter=1),  # rodar uma iteração por "epoch"
    loss=CrossEntropyLoss(),
    one_hot=True,
    callback=lambda weights, loss, step: enviar_circuito_por_epoca(
        qc.assign_parameters(weights), step
    ),
)

# Treianmento e Métricas

In [ ]:

EPOCHS = 3
print("\nTREINANDO VQC...")

for epoch in range(EPOCHS):
    vqc.fit(X_train, y_train)

    y_pred_val = vqc.predict(X_val)
    acc_val = accuracy_score(y_val, y_pred_val)
    print(f"[Epoch {epoch+1}] Accuracy validação: {acc_val:.4f}")

y_pred_test = vqc.predict(X_test)
print("\nRESULTADOS FINAIS NO TESTE:")
print("Acurácia:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
